ToDo:
- [X] Pick more than one card
- [ ] Modify print statement based on Arcana
    - Major *doesn't need* "the" infront of it
        - Don't want "the The Tower"
    - Minor does 
        - "the Ten of Pentacles" 
- [ ] Add meaning? 
- [ ] Set random seed to 42
- [ ] Record - day, time, and item? Put them into a CSV? & Auto Export?
- [ ] Set up a Cron Job to randomly run?


In [1]:
import pandas as pd
import random
from math import comb

In [4]:
csv_file_path = '/Users/jebbonanno/Documents/workspace/tarot_card_reader/tarot_cards.csv'

In [5]:
tarot_df = pd.read_csv(csv_file_path)

In [6]:
#probabilities 

prob_major_arcana = comb(22, 3) / comb(78, 3)
print(f"Probality all Major Arcana: {round((prob_major_arcana*100),2)}%")

prob_minor_arcana = 4*comb(14, 3) / comb(78, 3)
print(f"Probability all Minor Arcana: {round((prob_minor_arcana*100),2)}%")

prob_royals = comb(16, 3) / comb(78 , 3) 
print(f"Probability all Royals (Page, Knight, Queen, King): {round((prob_royals*100),2)}%")


Probality all Major Arcana: 2.02%
Probability all Minor Arcana: 1.91%
Probability all Royals (Page, Knight, Queen, King): 0.74%


### Three Card Pull

In [7]:
three_card_pull = tarot_df.sample(3).reset_index(drop=True)

for index, row in three_card_pull.iterrows():
    if index == 0:
        print(f"Past: {row['Name']}")
    elif index == 1:
        print(f"Present: {row['Name']}")
    elif index == 2:
        print(f"Future: {row['Name']}")
    else:
        print("Unexpected error")

# print()
        
# if three_card_pull['Suit'].nunique() == 1:
#     print("All values in 'Suit' are the same.")
# else:
#     print("Values in 'Suit' are different.")


Past: Two of Wands
Present: The Lovers
Future: Queen of Pentacles


In [ ]:
three_card_pull['Suit']

### Single Pull

In [8]:
def tarot_print_statement(card):
    if card[:3] == 'The':
        print(f"You drew {single_pick}.")
    else:
        print(f"You drew the {card}.")

In [9]:
tarot_print_statement(tarot_df['Name'].sample(1).values[0])

You drew the Ace of Cups.
